In [ ]:
import arcpy
import numpy as np
from scipy import io
import multiprocessing

def process_polygons(chunk):
    adjacent_counts = {}
    for oid, geom in chunk:
        adjacent_polygons = arcpy.SelectLayerByLocation_management(in_layer=fc_path,
                                                                   overlap_type="INTERSECT",
                                                                   select_features=geom.buffer(0.5),
                                                                   selection_type="NEW_SELECTION")
        adjacent_count = int(arcpy.GetCount_management(adjacent_polygons).getOutput(0))
        adjacent_counts[oid] = adjacent_count
    return adjacent_counts

if __name__ == "__main__":
    # Input feature class path
    fc_path = "BER_2024_Dublin_SPjoin_building.shp"
    
    # Create a spatial index for the feature class
    arcpy.management.AddSpatialIndex(fc_path)

    # Get the spatial reference of the feature class
    spatial_reference = arcpy.Describe(fc_path).spatialReference
    
    # Determine the number of CPU cores available
    num_cores = multiprocessing.cpu_count()

    # Create a search cursor to iterate through the polygons
    with arcpy.da.SearchCursor(fc_path, ["OID@", "SHAPE@"], spatial_reference=spatial_reference) as cursor:
        data = list(cursor)
        chunk_size = len(data) // num_cores
        chunks = [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

    # Process chunks in parallel
    with multiprocessing.Pool(processes=num_cores) as pool:
        results = pool.map(process_polygons, chunks)

    # Combine results from different processes
    adjacent_counts = {}
    for result in results:
        adjacent_counts.update(result)

    # Save variables to MATLAB format
    io.savemat('Nighbor.mat', {'adjacent_counts': adjacent_counts})


In [ ]:
import geopandas as gpd
Df=gpd.read_file('Dublinresidentials.shp')
from shapely.geometry import Point

# Create a buffer around each centroid with a radius of 100 meters
buffer_radius = 100  # in meters
Df['buffer'] = Df['centroid'].buffer(buffer_radius)
Df.to_file('Dublinresidentials.shp')

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union

# Read the input point shapefile

input_file = "BER_2024_Dublin.shp"
points_gdf = gpd.read_file(input_file)
print(points_gdf)
# Initialize group number
group_number = 1

# Initialize an empty GeoDataFrame to store grouped points
grouped_points = gpd.GeoDataFrame()

# Iterate over each point
while not points_gdf.empty:
    # Create buffer around each point
    buffers = points_gdf.geometry.buffer(300)
    
    # Select points within each buffer
    selected_points = gpd.GeoDataFrame()
    for buffer_geom in buffers:
        mask = points_gdf.geometry.within(buffer_geom)
        selected_points = selected_points.append(points_gdf[mask])
    
    # Identify points with the same value in the "A" column
    unique_values = selected_points['BER_RATING_HIGH'].unique()
    for value in unique_values:
        group_points = selected_points[selected_points['BER_RATING_HIGH'] == value]
        group_points['group'] = group_number
        
        # Add group points to the grouped points GeoDataFrame
        grouped_points = grouped_points.append(group_points)
        
        group_number += 1
            
    # Remove selected points from the original dataset
    points_gdf = points_gdf[~points_gdf.index.isin(selected_points.index)]

# Save the grouped points to a shapefile
grouped_points.to_file("grouped_points.shp")
